In [1]:
import pandas as pd
import numpy as np

# Define file paths
data_path = '../data/raw/readmission/'

# Read CSV files into DataFrames
admissions = pd.read_csv(data_path + 'admissions_202208161605.csv')
d_labitems = pd.read_csv(data_path + 'd_labitems_202208161605.csv')
diagnoses_icd = pd.read_csv(data_path + 'diagnoses_icd_202208161605.csv')
labevents = pd.read_csv(data_path + 'labevents_202208161605.csv')
patients = pd.read_csv(data_path + 'patients_202208161605.csv')
procedures_icd = pd.read_csv(data_path + 'procedures_icd_202208161605.csv')

In [2]:
# Heart failure ICD-9 codes
heart_codes = [
    '39891','40201','40211','40291','40401','40403','40411','40413',
    '40491','40493','4280','4281','42820','42821','42822','42823',
    '42830','42831','42832','42833','42840','42841','42842','42843','4289'
]

# Filter for HF diagnoses
hf_patients = diagnoses_icd[diagnoses_icd['icd9_code'].isin(heart_codes)][['subject_id', 'hadm_id']].drop_duplicates()

In [3]:
# Merge with admissions data
hf_data = hf_patients.merge(
    admissions,
    on=['subject_id', 'hadm_id'],
    how='left'
)

# Add patient demographics
hf_data = hf_data.merge(
    patients[['subject_id', 'gender', 'dob']],
    on='subject_id',
    how='left'
)

# Calculate age
hf_data['age'] = (pd.to_datetime(hf_data['admittime']).dt.year) - pd.to_datetime(hf_data['dob']).dt.year
hf_data = hf_data[hf_data['age'] >= 18]  # Adults only

In [4]:
# Sort by patient and admission time
hf_data = hf_data.sort_values(['subject_id', 'admittime'])

# Calculate next admission time
hf_data['next_admittime'] = hf_data.groupby('subject_id')['admittime'].shift(-1)

# Define target
hf_data['readmit_30'] = (
    (pd.to_datetime(hf_data['next_admittime']) - pd.to_datetime(hf_data['dischtime'])).dt.days <= 30
).astype(int)

# Exclude deaths
hf_data = hf_data[hf_data['deathtime'].isna()]

In [5]:
# Get key lab tests (BNP, Creatinine, Sodium)
lab_features = labevents.merge(
    d_labitems[d_labitems['label'].str.contains('BNP|Creatinine|Sodium', case=False)],
    on='itemid'
)

# Calculate mean values per admission
lab_means = lab_features.groupby(['hadm_id', 'label'])['valuenum'].mean().unstack()
lab_means.columns = [f'{col}_mean' for col in lab_means.columns]

# Merge with main data
hf_data = hf_data.merge(lab_means, on='hadm_id', how='left')

In [6]:
# Count non-HF diagnoses per admission
comorbidities = diagnoses_icd[~diagnoses_icd['icd9_code'].isin(heart_codes)]
hf_data['comorbidity_count'] = hf_data['hadm_id'].map(
    comorbidities.groupby('hadm_id')['icd9_code'].nunique()
).fillna(0)

In [7]:
print("Available columns in hf_data:", hf_data.columns.tolist())

Available columns in hf_data: ['subject_id', 'hadm_id', 'row_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'gender', 'dob', 'age', 'next_admittime', 'readmit_30', '24 hr Creatinine_mean', 'Albumin/Creatinine, Urine_mean', 'Amylase/Creatinine Ratio, Urine_mean', 'Creatinine_mean', 'Creatinine Clearance_mean', 'Creatinine, Ascites_mean', 'Creatinine, Body Fluid_mean', 'Creatinine, Joint Fluid_mean', 'Creatinine, Pleural_mean', 'Creatinine, Serum_mean', 'Creatinine, Urine_mean', 'NTproBNP_mean', 'Protein/Creatinine Ratio_mean', 'Sodium_mean', 'Sodium, Ascites_mean', 'Sodium, Body Fluid_mean', 'Sodium, Pleural_mean', 'Sodium, Stool_mean', 'Sodium, Urine_mean', 'Sodium, Whole Blood_mean', 'Urine Creatinine_mean', 'comorbidity_count']


In [8]:
# Check if BNP data exists in labevents
if 'BNP_mean' not in hf_data.columns:
    print("Attempting to fetch BNP data...")
    
    # Get BNP itemid(s)
    bnp_items = d_labitems[d_labitems['label'].str.contains('BNP', case=False)]['itemid']
    
    if not bnp_items.empty:
        # Calculate mean BNP per admission
        bnp_values = labevents[labevents['itemid'].isin(bnp_items)]
        bnp_means = bnp_values.groupby('hadm_id')['valuenum'].mean().rename('BNP_mean')
        hf_data = hf_data.merge(bnp_means, on='hadm_id', how='left')
        print("BNP_mean added successfully")
    else:
        print("Warning: No BNP tests found in d_labitems")
        hf_data['BNP_mean'] = np.nan  # Add as missing

Attempting to fetch BNP data...
BNP_mean added successfully


In [9]:
if 'comorbidity_count' not in hf_data.columns:
    print("Recalculating comorbidity count...")
    
    # Count non-HF diagnoses
    comorbidities = diagnoses_icd[~diagnoses_icd['icd9_code'].isin(heart_codes)]
    comorbidities = comorbidities[comorbidities['hadm_id'].isin(hf_data['hadm_id'])]
    comorbidity_counts = comorbidities.groupby('hadm_id')['icd9_code'].nunique().rename('comorbidity_count')
    
    hf_data = hf_data.merge(comorbidity_counts, on='hadm_id', how='left').fillna({'comorbidity_count': 0})
    print("comorbidity_count added successfully")

In [10]:
# Start with guaranteed basic features
features = ['age', 'gender', 'admission_type', 'insurance']

# Add available lab features
lab_features = ['Creatinine_mean', 'Sodium_mean', 'BNP_mean']
features += [f for f in lab_features if f in hf_data.columns]

# Add comorbidity count if available
if 'comorbidity_count' in hf_data.columns:
    features.append('comorbidity_count')

print("Final features being used:", features)

# Create final dataset
X = pd.get_dummies(hf_data[features], drop_first=True)
y = hf_data['readmit_30']

Final features being used: ['age', 'gender', 'admission_type', 'insurance', 'Creatinine_mean', 'Sodium_mean', 'BNP_mean', 'comorbidity_count']


In [11]:
# Check for missing values
print("\nMissing values per feature:")
print(X.isna().sum())

# Simple imputation (adjust as needed)
if X.isna().any().any():
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)


Missing values per feature:
age                            0
Creatinine_mean              124
Sodium_mean                  126
BNP_mean                    9574
comorbidity_count              0
gender_M                       0
admission_type_EMERGENCY       0
admission_type_URGENT          0
insurance_Medicaid             0
insurance_Medicare             0
insurance_Private              0
insurance_Self Pay             0
dtype: int64


In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Define feature groups for different imputation strategies
vital_features = ['age', 'comorbidity_count']
lab_features = ['Creatinine_mean', 'Sodium_mean', 'BNP_mean']
categoricals = [col for col in X.columns if col.startswith(('gender_', 'admission_type_', 'insurance_'))]

# Custom imputation: median for labs, 0 for BNP if >90% missing
imputer = SimpleImputer(
    strategy='median',
    missing_values=np.nan,
    add_indicator=True  # Adds binary columns indicating imputation
)

# If BNP is >90% missing, drop it and use creatinine as proxy
if X['BNP_mean'].isna().mean() > 0.9:
    print("Dropping BNP_mean due to excessive missingness")
    lab_features.remove('BNP_mean')
    X.drop(columns=['BNP_mean'], inplace=True)

# Scale only continuous features
preprocessor = ColumnTransformer(
    transformers=[
        ('vitals', 'passthrough', vital_features),
        ('labs', make_pipeline(imputer, StandardScaler()), lab_features),
        ('cats', 'passthrough', categoricals)
    ])

In [13]:
# Check for consistent lengths
assert len(X) == len(y), f"X has {len(X)} samples, y has {len(y)}"

# Check for NaN in target
print(f"Missing values in y: {y.isna().sum()}")

# Drop any rows with NaN in target if needed
if y.isna().any():
    y = y.dropna()
    X = X.loc[y.index]

Missing values in y: 0


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

# Define feature groups
numeric_features = ['age', 'Creatinine_mean', 'Sodium_mean', 'comorbidity_count']
categorical_features = [col for col in X.columns if col.startswith(('gender_', 'admission_type_', 'insurance_'))]

# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', 'passthrough', categorical_features)
    ])

# Create pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        class_weight='balanced_subsample',
        max_depth=5,
        random_state=42
    ))
])

# Verify pipeline
from sklearn.utils import estimator_checks
try:
    estimator_checks.check_estimator(pipeline)
    print("Pipeline validated successfully")
except Exception as e:
    print(f"Validation error: {str(e)}")

Validation error: Specifying the columns using strings is only supported for dataframes.


In [15]:
# Verify class balance
print(f"Class balance:\n{y.value_counts(normalize=True)}")

# Check for data leakage
print("\nMean values by class:")
print(X.assign(target=y).groupby('target').mean())

# Ensure no missing values in target
assert y.isna().sum() == 0, f"Missing values in y: {y.isna().sum()}"

Class balance:
readmit_30
0    0.919223
1    0.080777
Name: proportion, dtype: float64

Mean values by class:
              age  Creatinine_mean  Sodium_mean     BNP_mean  \
target                                                         
0       90.772377         1.702699   138.726474  6027.677798   
1       87.805986         2.018073   138.554589  6941.520382   

        comorbidity_count  gender_M  admission_type_EMERGENCY  \
target                                                          
0               13.003718  0.530879                  0.847737   
1               15.052632  0.546956                  0.929825   

        admission_type_URGENT  insurance_Medicaid  insurance_Medicare  \
target                                                                  
0                    0.031559            0.057677            0.738732   
1                    0.020640            0.063983            0.794634   

        insurance_Private  insurance_Self Pay  
target                         

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from xgboost import XGBClassifier

# Define feature types
numeric_features = ['age', 'Creatinine_mean', 'Sodium_mean', 'comorbidity_count']
categorical_features = [col for col in X.columns if col.startswith(('gender_', 'admission_type_', 'insurance_'))]

# Create preprocessor
preprocessor = ColumnTransformer([
    ('num', make_pipeline(
        SimpleImputer(strategy='median'),
        StandardScaler()
    ), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Simplified XGBoost without early stopping first
base_model = XGBClassifier(
    scale_pos_weight=3,  # Less aggressive weighting
    eval_metric='aucpr',
    max_depth=3,         # Shallower trees
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

# Initial pipeline
pipeline = make_pipeline(preprocessor, base_model)

In [18]:
from sklearn.model_selection import cross_validate

# Basic cross-validation
cv_results = cross_validate(
    pipeline, X, y,
    scoring=['precision', 'recall', 'f1'],
    cv=5,
    n_jobs=-1
)

print(f"\nCross-validation metrics:")
print(f"Precision: {cv_results['test_precision'].mean():.2f} ± {cv_results['test_precision'].std():.2f}")
print(f"Recall:    {cv_results['test_recall'].mean():.2f} ± {cv_results['test_recall'].std():.2f}")
print(f"F1:        {cv_results['test_f1'].mean():.2f} ± {cv_results['test_f1'].std():.2f}")


Cross-validation metrics:
Precision: 0.21 ± 0.13
Recall:    0.01 ± 0.01
F1:        0.02 ± 0.01


In [19]:
# Option A: If features > 20, use RFE
if len(numeric_features + categorical_features) > 20:
    from sklearn.feature_selection import RFE
    selector = RFE(
        estimator=RandomForestClassifier(n_estimators=50, random_state=42),
        n_features_to_select=15
    )
# Option B: Otherwise use variance threshold
else:
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold(threshold=0.01)  # Remove near-constant features

# Add to pipeline
pipeline.steps.insert(1, ('feature_selection', selector))

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, classification_report

# 1. Train-test split (already done above)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# 2. Fit preprocessor and feature selector on training data only
X_train_prep = preprocessor.fit_transform(X_train)
X_val_prep = preprocessor.transform(X_val)

feature_selector = SelectFromModel(RandomForestClassifier(n_estimators=50, random_state=42))
feature_selector.fit(X_train_prep, y_train)

X_train_fs = feature_selector.transform(X_train_prep)
X_val_fs = feature_selector.transform(X_val_prep)

# 3. Define and fit XGBoost with early stopping
model = XGBClassifier(
    scale_pos_weight=3,
    eval_metric='aucpr',
    max_depth=4,
    learning_rate=0.05,
    n_estimators=200,
    early_stopping_rounds=20,
    random_state=42
)
model.fit(
    X_train_fs, y_train,
    eval_set=[(X_val_fs, y_val)],
    verbose=10
)

# 4. Predict and evaluate
y_proba = model.predict_proba(X_val_fs)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, y_proba)
# Avoid division by zero in F1 calculation
with np.errstate(divide='ignore', invalid='ignore'):
    f1_scores = 2 * (precision * recall) / (precision + recall)
    f1_scores = np.nan_to_num(f1_scores, nan=0.0)  # Replace NaN with 0

optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]

print(f"\nF1-optimized threshold: {optimal_threshold:.3f}")
y_pred_optimal = (y_proba > optimal_threshold).astype(int)
print(classification_report(y_val, y_pred_optimal, zero_division=0))

# 5. Feature importances (with feature names)
try:
    feature_names = preprocessor.get_feature_names_out()
    selected_features = feature_names[feature_selector.get_support()]
except Exception as e:
    print(f"Could not extract feature names: {e}")
    selected_features = [f"Feature {i}" for i in range(len(model.feature_importances_))]

importances = model.feature_importances_
importances_df = pd.DataFrame({
    'feature': selected_features,
    'importance': importances
}).sort_values('importance', ascending=False)
print(importances_df.head(10))

[0]	validation_0-aucpr:0.11440
[10]	validation_0-aucpr:0.12238
[10]	validation_0-aucpr:0.12238
[20]	validation_0-aucpr:0.11982
[20]	validation_0-aucpr:0.11982
[30]	validation_0-aucpr:0.12043
[30]	validation_0-aucpr:0.12043

F1-optimized threshold: 0.234
              precision    recall  f1-score   support

           0       0.94      0.74      0.83      2206
           1       0.13      0.46      0.21       194

    accuracy                           0.72      2400
   macro avg       0.54      0.60      0.52      2400
weighted avg       0.87      0.72      0.78      2400

                  feature  importance
3  num__comorbidity_count    0.410726
1    num__Creatinine_mean    0.219519
0                num__age    0.185392
2        num__Sodium_mean    0.184363

F1-optimized threshold: 0.234
              precision    recall  f1-score   support

           0       0.94      0.74      0.83      2206
           1       0.13      0.46      0.21       194

    accuracy                      